In [1]:
!pip install mpi4py

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074467 sha256=d806e51e8f550c56d65477c0c9111bc6540c06ac2bdf444f20d2d69a16b45d67
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


# Question 1



In [2]:
%%writefile Q1.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
otherNum = numpy.zeros(1)

if rank == 0:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=1)
        comm.Recv(otherNum, source=1)
        print("Largest value was", otherNum[0])

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Recv(otherNum, source=0)
        comm.Send(max(randNum, otherNum), dest=0)
        

Writing Q1.py


In [3]:
!mpiexec --allow-run-as-root -n 2 python Q1.py

Process 0 drew the number 0.756810420114161
Process 1 drew the number 0.8995441131503884
Largest value was 0.8995441131503884


# Question 2




In [4]:
%%writefile Q2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
otherNum = numpy.zeros(1)

if rank == 0:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        derp = comm.Isend(randNum, dest=1)
        derp = comm.Irecv(otherNum, source=1)
        derp.wait()
        print("Largest value was", otherNum[0])

if rank == 1:
        derp = comm.Irecv(otherNum, source=0)
        derp.wait()
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        derp = comm.Isend(max(otherNum, randNum), dest=0)
        

Writing Q2.py


In [5]:
!mpiexec --allow-run-as-root -n 2 python Q2.py

Process 0 drew the number 0.6667270815454243
Process 1 drew the number 0.5057275529591572
Largest value was 0.6667270815454243


# Question 3



In [6]:
%%writefile Q3.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
maxNum = numpy.zeros(1)

randNum = numpy.random.random_sample(1)
print("Process", rank, "drew the number", randNum[0])
comm.Reduce(randNum, maxNum, op=MPI.MAX, root=0)


if rank == 0:
        print("Largest value was", maxNum[0])
        

Writing Q3.py


In [7]:
!mpiexec --allow-run-as-root -n 8 python Q3.py

Process 7 drew the number 0.46749389320707024
Process 6 drew the number 0.2870481548676421
Process 5 drew the number 0.6204079126623777
Process 3 drew the number 0.1972488577300925
Process 4 drew the number 0.05612189781165644
Process 1 drew the number 0.3467021250454799
Process 2 drew the number 0.3115894950130227
Process 0 drew the number 0.24088280421925468
Largest value was 0.6204079126623777


# Question 4

In [8]:
%%writefile Q4.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
otherNum = numpy.zeros(1)
status = MPI.Status()
processes = comm.Get_size()

randNum = numpy.random.random_sample(1)
print("Process", rank, "drew the number", randNum[0])
comm.Send(randNum, dest=0)

if rank == 0:
    maxNum = randNum[0]
    for process in range(processes):
        comm.Recv(otherNum, source=MPI.ANY_SOURCE, status=status)
        maxNum = max(otherNum[0], maxNum)
        status_location = status.source
    print("Largest value was", maxNum)


Writing Q4.py


In [9]:
!mpiexec --allow-run-as-root -n 10 python Q4.py

Process 1 drew the number 0.425409521352364
Process 7 drew the number 0.8684568653722213
Process 5 drew the number 0.44957942190150546
Process 0 drew the number 0.5910980251484841
Process 2 drew the number 0.3192805079974277
Process 3 drew the number 0.254643965542258
Process 4 drew the number 0.9164839170870888
Process 6 drew the number 0.8095482488324176
Process 8 drew the number 0.6521532547141672
Process 9 drew the number 0.3679695565048057
Largest value was 0.9164839170870888


# Question 5


In [10]:
%%writefile lab7q5.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


maxRandNum = numpy.zeros(10*size)
randNum = numpy.random.random_sample(10)
print("Process", rank, "drew the numbers", randNum)

if rank == 0:
  status = MPI.Status()
  maxRandNum[0:10] = randNum[0:10]
  comm.Probe(source=MPI.ANY_SOURCE, status=status)
  for process in range(size - 1):
    comm.Recv(maxRandNum[10*status.source:10*(status.source+1)], source=MPI.ANY_SOURCE, status=status)
    status_location = status.source
  print("Process", rank, "max random number", maxRandNum.max())
else:
  comm.Send(randNum, dest=0)


Writing lab7q5.py


In [11]:
!mpiexec --allow-run-as-root -n 8 python lab7q5.py

Process 3 drew the numbers [0.72614839 0.92073881 0.46494717 0.2820413  0.02090515 0.58624047
 0.47731064 0.97246223 0.42744781 0.9475484 ]Process 4 drew the numbers [0.3883259  0.9895733  0.51773761 0.42048592 0.93606454 0.53935274
 0.20734271 0.15156611 0.66006992 0.0713516 ]
Process 2 drew the numbers [0.14670525 0.24803233 0.45281297 0.91962613 0.1343755  0.32618859
 0.59909724 0.54632425 0.3792128  0.76660809]
Process 6 drew the numbers [0.68049794 0.21299394 0.41394625 0.52634704 0.18667088 0.11965787
 0.882688   0.96055679 0.75849023 0.33176872]
Process 0 drew the numbers [0.90893513 0.3963311  0.47926251 0.87934003 0.27358499 0.21627331
 0.07959085 0.2264409  0.6927828  0.03746301]
Process 1 drew the numbers [0.91510438 0.85859467 0.68480843 0.68222211 0.84023306 0.29870504
 0.83644827 0.24986032 0.5398788  0.44551866]

Process 7 drew the numbers [0.03005183 0.34341898 0.71358445 0.3671979  0.96481005 0.67626768
 0.72254065 0.70562252 0.60310336 0.0417217 ]
Process 5 drew the n

# Question 6

In [12]:
# LECTURE CODE

%%writefile conway_seq.py

import numpy, sys
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
P = comm.Get_size()

ROWS = 8
COLS = 8

def GETPOS(r, c):
  return r*COLS+c

def GETCOL(p):
  return p%COLS

def GETROW(p):
  return p//COLS

def D_LEFT(p):
  return p+COLS-1 if (GETCOL(p) == 0) else p-1

def D_RIGHT(p):
  return p-COLS+1 if (GETCOL(p) == COLS-1) else p+1

def D_TOP(p):
  return GETCOL(p)+((ROWS-1) * COLS) if (GETROW(p) == 0) else p-COLS

def D_BOTTOM(p):
  return GETCOL(p) if GETROW(p) == ROWS-1 else p+COLS
                     
def evolve_cell(inboard, outboard, position):
  count = inboard[D_LEFT(position)]
  count += inboard[D_RIGHT(position)]
  count += inboard[D_TOP(position)]
  count += inboard[D_BOTTOM(position)]
  count += inboard[D_LEFT(D_TOP(position))]
  count += inboard[D_RIGHT(D_TOP(position))]
  count += inboard[D_LEFT(D_BOTTOM(position))]
  count += inboard[D_RIGHT(D_BOTTOM(position))]

  outboard[position] = 1 if count == 3 or (inboard[position] and count == 2) else 0

def update_row(inboard, outboard, row):
  position = GETPOS(row, 0)
  end = position + COLS
  while position < end:
    evolve_cell(inboard, outboard, position)
    position += 1
    
    
def simulate(board, ticks, output):
  rows_per_p = ROWS // P
  status = MPI.Status()

  for i in range(ticks):
    # logic will alternate which board to read from / write to
    board_source = board[i%2]
    board_target = board[(i+1)%2]
    if output >= 2:
      print("board at tick %d:\n" % i)
      print_board(board_source)

    comm.Bcast(board_source, root=0)

    for j in range(rank*rows_per_p, (rank+1)*rows_per_p):
      update_row(board_source, board_target, j)

    if rank != 0:
      # send data to process 0
      comm.Send(board_target[rank*rows_per_p*COLS:(rank+1)*rows_per_p*COLS],dest=0)
    else:
      # receive data at process 0
      for j in range(P-1):
        # receive message
        comm.Probe(source=MPI.ANY_SOURCE, status=status)
        comm.Recv(board_target[status.source*rows_per_p*COLS:(status.source+1)*rows_per_p*COLS], source=status.source)
        # check the source of the message
        # write the received data to the correct target rows
        

def init_random(board):
  position = 0
  positions = ROWS * COLS
  while position < positions:
    board[position] = numpy.random.randint(2)
    position += 1
    
def init_glider(board):
  board[GETPOS(1,2)] = 1
  board[GETPOS(2,2)] = 1
  board[GETPOS(3,2)] = 1
  board[GETPOS(3,1)] = 1
  board[GETPOS(2,0)] = 1

    
def print_board(board):
  position = 0
  positions = ROWS * COLS
  while position < positions:
    print('x' if board[position] == 1 else ' ', end='')
    position += 1
    if position % COLS == 0:
      print()
    
    
def main():
  """
  Conway's game of life simulation. Optional arguments (must provide in order)
  GRID ticks initialization output
  GRID = number of rows and columns in the square grid
  
  example: 8 100 g 1
  would run with an 8x8 grid for 100 simulation ticks and only output the final
  
  """
  output = 1 # set to 2 to debug
  ticks = 10
  init = 'g' # glider
  if len(sys.argv) >= 2:
    global ROWS, COLS
    ROWS = int(sys.argv[1])
    COLS = ROWS
  if len(sys.argv) >= 3:
    ticks = int(sys.argv[2])
  if len(sys.argv) >= 4:
    init = sys.argv[3]
  if len(sys.argv) >= 5:
    output = int(sys.argv[4])
    
  board = [numpy.zeros(ROWS*COLS), numpy.zeros(ROWS*COLS)]
  if rank == 0:
    time_start = MPI.Wtime()
    
    if init == 'g':
      init_glider(board[0])
    else:
      init_random(board[0])
    simulate(board, ticks, output)
    
    time_end = MPI.Wtime()
    
    if output >= 1:
      print("final board:\n")
      print_board(board[0])
    print("simulation took %f seconds" % (time_end-time_start))
  else:
    simulate(board, ticks, output)
  
if __name__ == "__main__":
  main()

Writing conway_seq.py


In [13]:
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 16 20 g 1
!mpiexec --allow-run-as-root -n 2 python conway_seq.py 150 200 g 1
!mpiexec --allow-run-as-root -n 8 python conway_seq.py 150 200 g 1
!mpiexec --allow-run-as-root -n 16 python conway_seq.py 150 200 g 1

final board:

                
                
                
                
                
                
       x        
     x x        
      xx        
                
                
                
                
                
                
                
simulation took 0.024446 seconds
final board:

                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                               

In [14]:
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 32 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 64 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 128 1000 r 0
!mpiexec --allow-run-as-root -n 1 python conway_seq.py 256 1000 r 0

simulation took 5.385006 seconds
simulation took 21.487580 seconds
simulation took 82.905084 seconds
simulation took 321.052634 seconds
